In [7]:
import cv2
import numpy as np
import tensorflow as tf
import json
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

model = tf.keras.models.load_model("modelEncoderAtentionLarge0.018%_Margin_0.2.m5/")

all_samples_product  = json.load(open("data.json","r"))["data"]

my_face = np.asarray(cv2.resize(cv2.imread("detcted.jpg"),dsize=(32,32), interpolation = cv2.INTER_CUBIC))
faces = []
for i in all_samples_product[:10000]:
    faces+=[np.asarray(cv2.resize(cv2.imread(path),dsize=(32,32), interpolation = cv2.INTER_CUBIC),dtype=np.float32) for path in i]

faces.append(my_face)

In [ ]:
faces

In [9]:

dimo_data = model.predict(np.array(faces))

938/938 [==============================] - 5s 5ms/step


In [10]:
dimo_data.shape

(30001, 1024)

In [11]:
# define a video capture object
vid = cv2.VideoCapture(0)




while(True):

    # Capture the video frame
    # by frame

    ret, frame = vid.read()
    grayed = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(grayed, 1.1, 4)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        face = frame[y:y+h,x:x+w]
        img2= cv2.resize(face,dsize=(32,32), interpolation = cv2.INTER_CUBIC)
        #Numpy array
        np_image_data = np.asarray([img2],dtype=np.float32)
        encodedd = model.predict(np_image_data,verbose=0)
        val2 = [np.abs((encodedd-i).sum())  for i in dimo_data]
        val1 = np.abs((encodedd-dimo_data[-1]).sum())
        if val1!=val2[-1]:
            print("######################\nbazinga\n######################")
        #cv2.imwrite('detcted2q.jpg', img2)
    # Display the resulting frame
    cv2.imshow('frame', frame)

    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# After the loop release the cap object
vid.release()
# Destroy alql the windows
cv2.destroyAllWindows()